In [1]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister

In [2]:
NUM_ELEMENTS = 5
NUM_QUBITS = (NUM_ELEMENTS - 1).bit_length()

In [3]:
def init_perm_reg(perm_reg: QuantumRegister, num_elements: int):
    qc: QuantumCircuit = QuantumCircuit(perm_reg)
    num_qubits: int = len(perm_reg) // num_elements
    for k in range(1, num_elements):
        for i in range(k.bit_length()):
            if (k >> i) & 1:
                qc.x(perm_reg[k * num_qubits + i])
    return qc

In [4]:
def shukla_vedula(qreg: QuantumRegister, num_states: int):
    qcircuit: QuantumCircuit = QuantumCircuit(qreg)
    num_qubits: int = (num_states - 1).bit_length()
    if num_qubits != num_states.bit_length():  # num_states is a power of 2
        qcircuit.h(qreg[:num_qubits])
        return qcircuit
    
    bit_pos: list[int] = [index for (index, bit) in enumerate(np.binary_repr(num_states)[::-1]) if bit == '1']
    qcircuit.x(qreg[bit_pos[1:num_states.bit_length()]])
    cur_num_states: int = 1 << bit_pos[0]
    theta: float = -2 * np.arccos(np.sqrt(cur_num_states / num_states))
    if bit_pos[0] > 0:  # num_states is even
        qcircuit.h(qreg[0:bit_pos[0]])
    qcircuit.ry(theta, qreg[bit_pos[1]])
    qcircuit.ch(qreg[bit_pos[1]], qreg[bit_pos[0]:bit_pos[1]], ctrl_state='0')
    for m in range(1, len(bit_pos) - 1):
        theta = -2 * np.arccos(np.sqrt(2**bit_pos[m] / (num_states - cur_num_states)))
        qcircuit.cry(theta, qreg[bit_pos[m]], qreg[bit_pos[m + 1]], ctrl_state='0')
        qcircuit.ch(qreg[bit_pos[m + 1]], qreg[bit_pos[m]:bit_pos[m + 1]], ctrl_state='0')
        cur_num_states += 1 << bit_pos[m]
    return qcircuit

In [7]:
def disentangling_state_gen(perm_reg: QuantumRegister, ancilla_reg: QuantumRegister, num_elements: int, disentangling: bool = True):
    num_qubits: int = (num_elements - 1).bit_length()
    qc: QuantumCircuit = QuantumCircuit(perm_reg, ancilla_reg)
    qc.compose(init_perm_reg(perm_reg, num_elements), inplace=True)
    offset: int = 0
    for i in range(1, num_elements):
        qc.barrier(perm_reg, ancilla_reg)
        num_ctrl: int = i.bit_length()
        qc.compose(other=shukla_vedula(ancilla_reg[offset:offset + num_ctrl], i + 1), qubits=ancilla_reg, inplace=True)
        qc.barrier(perm_reg, ancilla_reg)
        for j in range(0, i):
            for q in range(0, num_qubits):
                qc.mcx([perm_reg[j * num_qubits + q]] + ancilla_reg[offset:offset + num_ctrl], perm_reg[i * num_qubits + q], ctrl_state=np.binary_repr(j, num_ctrl) + '1')
                qc.mcx([perm_reg[i * num_qubits + q]] + ancilla_reg[offset:offset + num_ctrl], perm_reg[j * num_qubits + q], ctrl_state=np.binary_repr(j, num_ctrl) + '1')
                qc.mcx([perm_reg[j * num_qubits + q]] + ancilla_reg[offset:offset + num_ctrl], perm_reg[i * num_qubits + q], ctrl_state=np.binary_repr(j, num_ctrl) + '1')
        qc.barrier(perm_reg, ancilla_reg)
        if disentangling:
            for j in range(1, i + 1):
                for k in range(j.bit_length()):
                    if (j >> k) & 1:
                        qc.mcx(control_qubits=perm_reg[j * NUM_QUBITS:(j + 1) * NUM_QUBITS], target_qubit=ancilla_reg[offset + k], ctrl_state=i)
        else:
            offset += i.bit_length()
    return qc